In [ ]:
here = '2_Application'

In [3]:
import os
import pickle
import pandas as pd
import numpy as np

## Define Vanilla BERT Encoder

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.cluster import KMeans
from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from string import punctuation
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained(
    'bert-base-uncased',
    output_hidden_states=True,
    return_dict=True
)

###############################
# 2. Preprocessing function
###############################
def preprocess_text(text):
    import unicodedata

    if pd.isnull(text) or text.strip() == "":
        return None

    # Normalize Unicode (remov accents, special characters)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Remove redundant whitespace
    text = " ".join(text.split())

    return text.strip()

###############################
# 3. BERT embedding function
###############################
def get_bert_embedding(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Use GPU
    model.to(device)

    # This function should rely on a loaded tokenizer & model with hidden_states=True
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, return_dict=True)
        # second-to-last layer
        second_last_layer = outputs.hidden_states[-2]  # shape: [batch_size, seq_len, hidden_size]
        embeddings = second_last_layer.mean(dim=1)     # mean pooling
    return embeddings.cpu().numpy().flatten()

###############################
# 4. Cosine similarity function
###############################
def compute_similarity(emb1, emb2):
    emb1_normalized = torch.nn.functional.normalize(emb1, p=2, dim=1)
    emb2_normalized = torch.nn.functional.normalize(emb2, p=2, dim=1)
    return torch.mm(emb1_normalized, emb2_normalized.t())

#### Get base embeddings

def get_base_embs(phrase, adjective_list):
  base_embs = {}
  for adj in adjective_list:
    base_sentence = phrase.replace("...", f"{adj}")
    base_emb = get_bert_embedding(base_sentence)
    base_embs[adj] = base_emb
  return base_embs


tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained(
    'bert-base-uncased',
    output_hidden_states=True,
    return_dict=True
)

## Define Other Encoders

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
tqdm.pandas()

models = {
    'sbert': SentenceTransformer('sentence-transformers/all-mpnet-base-v2'),
    #'qwen': SentenceTransformer("Qwen/Qwen3-Embedding-4B")
}

def encode(models, name, docs):
  """
  Input models (dict), model (name), docs (pd.Series)
  Returns embeddings (list)
  """

  if name == 'bert':
    processed = docs.apply(preprocess_text)
    return processed.progress_apply(get_bert_embedding)

  else:
    return models[name].encode(docs, show_progress_bar=True).tolist()

# US Responses

In [6]:
with open(os.path.join(here, 'data/us_dict.pkl'), 'rb') as f:
  us = pickle.load(f)

us = pd.DataFrame(us)

In [7]:
us['BERT'] = encode(models, 'bert', us['Text'])

100%|██████████| 2656/2656 [00:27<00:00, 97.38it/s] 


In [9]:
us_dict = us.to_dict(orient='records')

In [ ]:
with open(os.path.join(here, 'data/us_dict_embedded.pkl'), 'wb') as f:
  us = pickle.dump(us_dict, f)